In [9]:
import os
# os.environ["MODIN_ENGINE"] = "dask"
# import modin.pandas as pd
import pandas as pd
import dask
import dask.dataframe as dd
import numpy as np
from typing import List, Callable, Dict
from realgam.quantlib import general_utils as gu
from realgam.quantlib.engineer.op_engineer import OpEngineer, GroupOpEngineer
from realgam.quantlib.engineer.ta_engineer import TalibEngineer, GroupTalibEngineer
from realgam.quantlib.engineer.alpha_engineer import AlphaEngineer
from realgam.quantlib.engineer.interface import BaseEngineer, GroupBaseEngineer
import datetime

import time
PROJECT_PATH = os.getenv('QuantSystemMVP')
DATA_PATH = f'{PROJECT_PATH}/Data/stock_hist.obj'

In [26]:
stocks_df, stocks_extended_df, available_tickers = gu.load_file(DATA_PATH)

In [30]:
stacked_hist = stocks_df.copy()

In [31]:
stacked_hist

,ticker,open,high,low,close,openadj,highadj,lowadj,closeadj,volume
date,,,,,,,,,,
2012-01-03,A,35.84,36.990,35.740,36.48,23.651649,24.410561,23.585657,24.074,2973100.0
2012-01-04,A,36.15,36.360,35.590,36.19,23.855604,23.994184,23.486056,23.882,3327500.0
2012-01-05,A,35.88,37.320,35.720,37.00,23.677891,24.628174,23.572304,24.417,4894600.0
2012-01-06,A,37.09,37.555,36.805,37.40,24.476425,24.783288,24.288348,24.681,3370100.0
2012-01-09,A,37.62,38.420,37.472,38.38,24.826455,25.354397,24.728786,25.328,3168500.0
...,...,...,...,...,...,...,...,...,...,...
2013-03-11,ZZ,2.20,2.210,2.200,2.20,2.200000,2.210000,2.200000,2.200,407100.0
2013-03-12,ZZ,2.20,2.210,2.200,2.21,2.200000,2.210000,2.200000,2.210,160000.0
2013-03-13,ZZ,2.20,2.210,2.200,2.21,2.200000,2.210000,2.200000,2.210,308900.0


In [32]:
stacked_hist = dd.from_pandas(stacked_hist, npartitions=10)

In [36]:
stacked_hist.compute()

,ticker,open,high,low,close,openadj,highadj,lowadj,closeadj,volume
date,,,,,,,,,,
2012-01-03,A,35.840,36.990,35.740,36.480,23.651649,24.410561,23.585657,24.074,2973100.0
2012-01-03,RINO,0.030,0.030,0.030,0.030,0.030000,0.030000,0.030000,0.030,500.0
2012-01-03,STRZB,77.950,77.950,77.950,77.950,9.841000,9.841000,9.841000,9.841,0.0
2012-01-03,KLAC,48.720,49.290,47.390,47.460,30.297558,30.652024,29.470469,29.514,3310800.0
2012-01-03,ENTR1,5.280,5.500,5.240,5.500,5.280000,5.500000,5.240000,5.500,1221655.0
...,...,...,...,...,...,...,...,...,...,...
2022-03-16,SRL,8.160,8.650,7.920,8.000,8.160000,8.650000,7.920000,8.000,38570.0
2022-03-16,INN,9.950,10.100,9.800,10.060,9.950000,10.100000,9.800000,10.060,728306.0
2022-03-16,INMD,36.147,37.500,35.529,37.270,36.147000,37.500000,35.529000,37.270,2094707.0


In [33]:
tickers = list(stacked_hist.ticker.unique()[:1000])
tripA = stacked_hist[stacked_hist.ticker.isin(tickers)]
A = stacked_hist[stacked_hist.ticker.isin(['A'])]

In [39]:
tripA.groupby('ticker').apply(lambda df_g: df_g.sort_values('date')['volume'].rolling(5).mean(), meta=('volume', 'f8')).compute()

ticker  date      
AAME    2012-01-03         NaN
        2012-01-04         NaN
        2012-01-05         NaN
        2012-01-06         NaN
        2012-01-09      2360.0
                        ...   
ZEUS    2022-03-10    249624.2
        2022-03-11    191535.2
        2022-03-14    152770.2
        2022-03-15    132354.4
        2022-03-16    105195.6
Name: volume, Length: 1926216, dtype: float64

In [42]:
s_time_chunk = time.time()
dummy = tripA.groupby('ticker').apply(lambda df_g: df_g.sort_values('date')['volume'].rolling(5).apply(np.argmin, engine='cython', raw=True), meta=('volume', 'f8')).compute()
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")


Total time: 11.525414228439331 sec


In [17]:
goe = GroupOpEngineer(tripA, 'ticker')


NotImplementedError: Dataframe only supports sorting by a single column which must be passed as a string or a list of a single string.
You passed ['ticker', 'date']

In [35]:
A.compute()

,ticker,open,high,low,close,openadj,highadj,lowadj,closeadj,volume
date,,,,,,,,,,
2012-01-03,A,35.84,36.990,35.740,36.48,23.651649,24.410561,23.585657,24.074,2973100.0
2012-01-04,A,36.15,36.360,35.590,36.19,23.855604,23.994184,23.486056,23.882,3327500.0
2012-01-05,A,35.88,37.320,35.720,37.00,23.677891,24.628174,23.572304,24.417,4894600.0
2012-01-06,A,37.09,37.555,36.805,37.40,24.476425,24.783288,24.288348,24.681,3370100.0
2012-01-09,A,37.62,38.420,37.472,38.38,24.826455,25.354397,24.728786,25.328,3168500.0
...,...,...,...,...,...,...,...,...,...,...
2022-03-10,A,131.40,134.520,131.290,134.33,131.400000,134.520000,131.290000,134.330,1933127.0
2022-03-11,A,135.43,136.440,130.160,130.31,135.430000,136.440000,130.160000,130.310,1815743.0
2022-03-14,A,131.00,132.260,126.900,127.58,131.000000,132.260000,126.900000,127.580,1737099.0


In [22]:
eng_values = tripA.groupby('ticker')['volume'].rolling('10D').apply(np.argmin, engine='cython', raw=True).add(
            1)

In [24]:
tripA['ts_argmin10_volume'] = eng_values.values

In [25]:
tripA.compute()

,ticker,open,high,low,close,openadj,highadj,lowadj,closeadj,volume,ts_argmin10_volume
date,,,,,,,,,,,
2012-01-03,A,35.84,36.990,35.740,36.480,23.651649,24.410561,23.585657,24.074,2973100.0,1.0
2012-01-03,RINO,0.03,0.030,0.030,0.030,0.030000,0.030000,0.030000,0.030,500.0,1.0
2012-01-03,STRZB,77.95,77.950,77.950,77.950,9.841000,9.841000,9.841000,9.841,0.0,1.0
2012-01-03,KLAC,48.72,49.290,47.390,47.460,30.297558,30.652024,29.470469,29.514,3310800.0,1.0
2012-01-03,ENTR1,5.28,5.500,5.240,5.500,5.280000,5.500000,5.240000,5.500,1221655.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2022-03-16,CRUS,80.99,83.850,80.220,83.810,80.990000,83.850000,80.220000,83.810,338406.0,4.0
2022-03-16,CRVL,160.60,165.870,159.420,165.510,160.600000,165.870000,159.420000,165.510,46305.0,5.0
2022-03-16,VIVO,25.04,25.800,24.925,25.750,25.040000,25.800000,24.925000,25.750,400748.0,8.0


In [5]:
s_time_chunk = time.time()
goe.ts_rank('volume', 7, inplace=True)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")


Total time: 11.078994989395142 sec


In [6]:
s_time_chunk = time.time()
goe.ts_corr('volume', 'lowadj', 7, inplace=True)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")


Total time: 1.670530080795288 sec


In [7]:
s_time_chunk = time.time()
goe.ts_argmin('volume', 7, inplace=True)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 1.9038870334625244 sec


In [8]:
s_time_chunk = time.time()
goe.cs_pctrank('volume', inplace=True)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 0.27015209197998047 sec


In [16]:
goe.df

,ticker,open,high,low,close,openadj,highadj,lowadj,closeadj,volume,cs_pctrank_volume,ts_rank7_volume,ts_corr7_volume_lowadj,ts_argmin7_volume
date,,,,,,,,,,,,,,
2012-01-03,A,35.84,36.990,35.740,36.48,23.651649,24.410561,23.585657,24.074,2973100.0,0.917184,NaN,NaN,NaN
2012-01-04,A,36.15,36.360,35.590,36.19,23.855604,23.994184,23.486056,23.882,3327500.0,0.927536,NaN,NaN,NaN
2012-01-05,A,35.88,37.320,35.720,37.00,23.677891,24.628174,23.572304,24.417,4894600.0,0.952381,NaN,NaN,NaN
2012-01-06,A,37.09,37.555,36.805,37.40,24.476425,24.783288,24.288348,24.681,3370100.0,0.925466,NaN,NaN,NaN
2012-01-09,A,37.62,38.420,37.472,38.38,24.826455,25.354397,24.728786,25.328,3168500.0,0.929607,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-10,AXH,9.98,9.980,9.980,9.98,9.980000,9.980000,9.980000,9.980,3374.0,0.153153,1.0,-0.001595,3.0
2022-03-11,AXH,9.98,9.980,9.980,9.98,9.980000,9.980000,9.980000,9.980,90.0,0.043011,1.0,-0.001449,2.0
2022-03-14,AXH,9.98,9.980,9.980,9.98,9.980000,9.980000,9.980000,9.980,446131.0,0.587537,6.0,-0.000061,1.0


In [42]:
goe0 = GroupOpEngineer(A, 'ticker')

In [7]:
goe0.ts_rank('volume', 3, inplace=True)

In [11]:
dummy = A.iloc[:, 1:]

In [44]:
tripA1 = tripA[['ticker','volume']]

In [45]:
ofe0 = GroupOpEngineer(tripA1, 'ticker')

In [53]:
s_time_chunk = time.time()
dummy = tripA1.groupby('ticker').volume.rolling(10).apply(rank_last_value, raw=True)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")


Total time: 5.396774768829346 sec


In [55]:
tripA['rank'] = dummy.values

In [56]:
tripA

,ticker,open,high,low,close,openadj,highadj,lowadj,closeadj,volume,rank
date,,,,,,,,,,,
2012-01-03,A,35.84,36.990,35.740,36.48,23.651649,24.410561,23.585657,24.074,2973100.0,NaN
2012-01-04,A,36.15,36.360,35.590,36.19,23.855604,23.994184,23.486056,23.882,3327500.0,NaN
2012-01-05,A,35.88,37.320,35.720,37.00,23.677891,24.628174,23.572304,24.417,4894600.0,NaN
2012-01-06,A,37.09,37.555,36.805,37.40,24.476425,24.783288,24.288348,24.681,3370100.0,NaN
2012-01-09,A,37.62,38.420,37.472,38.38,24.826455,25.354397,24.728786,25.328,3168500.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2017-10-02,AMFW,6.66,6.680,6.630,6.66,6.660000,6.680000,6.630000,6.660,12407.0,0.0
2017-10-03,AMFW,6.64,6.820,6.580,6.67,6.640000,6.820000,6.580000,6.670,1651903.0,9.0
2017-10-04,AMFW,6.72,6.920,6.680,6.87,6.720000,6.920000,6.680000,6.870,415055.0,8.0


In [52]:
s_time_chunk = time.time()
dummy = tripA1.groupby('ticker').volume.rolling(10).apply(lambda x: pd.Series(x).rank().iloc[-1])
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")


KeyboardInterrupt: 

In [46]:
s_time_chunk = time.time()
ofe0.ts_rank('volume', 10, inplace=True)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 80.64372301101685 sec


In [47]:
tripA2 = tripA[['ticker', 'volume']].reset_index()
tripA2 = tripA2.pivot(index='date', columns='ticker', values= 'volume')

In [48]:
s_time_chunk = time.time()
dummy = tripA2.rolling(10).apply(lambda x: pd.Series(x).rank().iloc[-1], engine='cython', raw=True)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 80.60853600502014 sec


In [23]:
tripA2 = tripA[['ticker', 'volume']].reset_index()

In [25]:
tripA2 = tripA2.pivot(index='date', columns='ticker', values= 'volume')

In [26]:
tripA2

ticker,A,AA,AAP
date,,,
2012-01-03,2973100.0,NaN,786600.0
2012-01-04,3327500.0,NaN,1412800.0
2012-01-05,4894600.0,NaN,596900.0
2012-01-06,3370100.0,NaN,1346600.0
2012-01-09,3168500.0,NaN,686100.0
...,...,...,...
2022-03-10,1933127.0,10227270.0,613411.0
2022-03-11,1815743.0,5687948.0,527477.0
2022-03-14,1737099.0,7906373.0,1229028.0


In [4]:
s_time_chunk = time.time()
afe0.alpha2(2, 6, inplace=True)
e_time_chunk = time.time()

print(f"Total time: {e_time_chunk - s_time_chunk} sec")

/Users/punlee/miniconda3/envs/quant/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Total time: 4843.606370210648 sec
